<a href="https://colab.research.google.com/github/laurinwagner/grouploss/blob/master/grouploss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/laurinwagner/grouploss.git


Cloning into 'grouploss'...
remote: Enumerating objects: 152, done.
remote: Total 152 (delta 0), reused 0 (delta 0), pack-reused 152
Receiving objects: 100% (152/152), 572.29 MiB | 11.84 MiB/s, done.
Resolving deltas: 100% (42/42), done.
Checking out files: 100% (51/51), done.


In [1]:

!pip uninstall torchvision==0.5.0
!pip uninstall  torch==1.4.0
!pip uninstall pillow
!pip install pillow==6.2.1
!pip install torchvision==0.2.0
!pip install  torch==1.1.0 


Uninstalling torchvision-0.5.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/torchvision-0.5.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torchvision/*
Proceed (y/n)? y
  Successfully uninstalled torchvision-0.5.0
Uninstalling torch-1.4.0:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.6/dist-packages/caffe2/*
    /usr/local/lib/python3.6/dist-packages/torch-1.4.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torch/*
Proceed (y/n)? y
y
  Successfully uninstalled torch-1.4.0
Uninstalling Pillow-7.0.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/PIL/*
    /usr/local/lib/python3.6/dist-packages/Pillow-7.0.0.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled Pillow-7.0.0
     |████████████████████████████████| 2.1MB 2.7MB/s 
ERROR: fastai 1.0.60 requires torch>=1.0.0, which is not installed.
ERROR: fastai 1.0.60 requires torchvision, which is not installed.

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 753.4MB 20kB/s 
     |████████████████████████████████| 676.9MB 21kB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0


In [0]:
%%bash
git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./


Process is interrupted.


mount drive to pull dataset

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


get dataset and put it in correct directory

In [6]:
%%bash
mkdir grouploss/datasets
cp 'gdrive/My Drive/grouploss/datasets/CARS.zip' 'grouploss/datasets'
cd grouploss/datasets
unzip CARS.zip

Archive:  CARS.zip
   creating: CARS/
   creating: CARS/images/
   creating: CARS/images/001/
  inflating: CARS/images/001/000001.jpg  
  inflating: CARS/images/001/000002.jpg  
  inflating: CARS/images/001/000003.jpg  
  inflating: CARS/images/001/000004.jpg  
  inflating: CARS/images/001/000005.jpg  
  inflating: CARS/images/001/000006.jpg  
  inflating: CARS/images/001/000007.jpg  
  inflating: CARS/images/001/000008.jpg  
  inflating: CARS/images/001/000009.jpg  
  inflating: CARS/images/001/000010.jpg  
  inflating: CARS/images/001/000011.jpg  
  inflating: CARS/images/001/000012.jpg  
  inflating: CARS/images/001/000013.jpg  
  inflating: CARS/images/001/000014.jpg  
  inflating: CARS/images/001/000015.jpg  
  inflating: CARS/images/001/000016.jpg  
  inflating: CARS/images/001/000017.jpg  
  inflating: CARS/images/001/000018.jpg  
  inflating: CARS/images/001/000019.jpg  
  inflating: CARS/images/001/000020.jpg  
  inflating: CARS/images/001/000021.jpg  
  inflating: CARS/images

In [7]:
cd grouploss

/content/grouploss


In [8]:
#@title dirty fix for division by zero in dynamics.py
%%writefile dynamics.py

import torch
import warnings

# import logging
# logger = logging.getLogger('logger')
# logger.warning('prova')

__all__ = ['dynamics']


def dynamics(W, X, tol=1e-6, max_iter=5, mode='replicator', **kwargs):
    """
    Selector for dynamics
    Input:
    W:  the pairwise nxn similarity matrix (with zero diagonal)
    X:  an (n,m)-array whose rows reside in the n-dimensional simplex. If
        an (n,)-array is provided, it must reside in the n-dimensional simplex
    tol:  error tolerance
    max_iter:  maximum number of iterations
    mode: 'replicator' to run classical replicator dynamics,
          'exponential' to run exponential replicator dynamics.
          'inf_imm' to run infection-immunization dynamics (not implemented).
    """

    if mode == 'replicator':
        X = _replicator(W, X, tol, max_iter)
    elif mode == 'exponential':
        X = _exponential(W, X, tol, max_iter, kwargs.get('k', 1.))
    elif mode == 'inf_imm':
        if X.dim == 2:
            raise ValueError('Currently, only one-dimensional vectors are '
                             'accepted with \'inf_imm\' mode')
        X = _inf_imm(W, X, tol, max_iter)
    else:
        raise ValueError('mode \'' + mode + '\' is not defined.')

    return X


def _replicator(W, X, tol, max_iter):
    """
    Replicator Dynamics
    Output:
    X:  the population(s) at convergence
    i:  the number of iterations needed to converge
    prec:  the precision reached by the dynamical system
    """

    i = 0
    while i < max_iter:
        X = X * torch.matmul(W, X)
        # z = X.register_hook(lambda g: print(g))
        # print(z)
        X /= X.sum(dim=X.dim() - 1).unsqueeze(X.dim() - 1)+1e-12 # dirty fix for division of zero error creating Nan
        # z = X.register_hook(lambda g: print(g))
        # print(z)

        i += 1

    return X


def _exponential(W, X, tol, max_iter, k):
    """
    Exponential Replicator Dynamics
    Input:
    k: the "acceleration" parameter of the dynamical system
    Output:
    x:  the population(s) at convergence
    iter:  the number of iterations needed to converge
    prec:  the precision reached by the dynamical system
    """

    err = 2. * tol
    i = 0
    while err > tol and i < max_iter:
        X_old = X
        X = X * torch.exp(k * (torch.matmul(W, X) - 1.))  # softmax trick
        X /= X.sum(dim=X.dim() - 1).unsqueeze(X.dim() - 1)

        err = torch.norm(X - X_old)
        i += 1

    if i == max_iter:
        warnings.warn("Maximum number of iterations reached.")

    return X


def _inf_imm(W, X, tol, max_iter):
    """
    Infection Immunization Dynamics
    Output:
    x:  the population(s) at convergence
    iter:  the number of iterations needed to converge
    prec:  the precision reached by the dynamical system
    """
    dtype = X.dtype  # casting dtype for ByteTensors

    WX = torch.matmul(W, X)
    XWX = torch.matmul(WX, X)
    r = WX - XWX

    # TODO: check Nash error
    err = (torch.max(X, r) ** 2.).sum()
    i = 0

    while err > tol and i < max_iter:
        max_, imax = torch.max(r, dim=0)
        min_, imin = torch.min(r * (X > 0.).to(dtype), dim=0)  # TODO: check
        infective = imax if max_ > -min_ else imin
        den = W[infective, infective] - WX[infective] - r[infective]

        do_remove = False
        if r[infective] >= 0.:
            mu = 1.
            if den < 0:
                opt_delta = -r[infective] / den
                if opt_delta < mu:
                    mu = opt_delta
                # if mu < 0.: mu = 0.
        else:
            do_remove = True
            mu = X[infective] / (X[infective] - 1.)
            if den < 0.:
                opt_delta = -r[infective] / den
                if opt_delta >= mu:
                    mu = opt_delta
                    do_remove = False

        n = X.shape[0]
        X = mu * ((torch.arange(n, device=X.device) == infective).to(
            dtype) - X) + X

        if do_remove:
            X[infective] = 0.

        WX = mu * (W[infective, :] - WX) + WX

        XWX = torch.matmul(X, WX)
        r = WX - XWX

        err = (torch.max(X, r) ** 2.).sum()
        i += 1

    if i == max_iter:
        warnings.warn("Maximum number of iterations reached.")

    return X

Overwriting dynamics.py


In [9]:
#@title dont load finteund model for embed in load_trained_net.py
%%writefile net/load_trained_net.py
import torch
import os
import net
import torch.nn as nn


def load_net(dataset, net_type, nb_classes, embed=False, sz_embedding=512):

    if net_type == 'bn_inception':
        model = net.bn_inception(pretrained=True)
        model.last_linear = nn.Linear(1024, nb_classes)
        model.load_state_dict(torch.load('net/finetuned_' + dataset + '_' + net_type + '.pth'))

        if embed:
            model = net.Inception_embed(model, 1024, sz_embedding, num_classes=nb_classes)
            #model.load_state_dict(torch.load(os.path.join('net', 'finetuned_cub_embedded_512_10_.pth')))

    elif net_type == 'resnet18':
        model = net.resnet18(pretrained=True)
        model.fc = nn.Linear(512, nb_classes)
        model.load_state_dict(torch.load('net/finetuned_' + dataset + '_' + net_type + '.pth'))

    elif net_type == 'resnet34':
        model = net.resnet34(pretrained=True)
        model.fc = nn.Linear(512, nb_classes)
        model.load_state_dict(torch.load('net/finetuned_' + dataset + '_' + net_type + '.pth'))

    elif net_type == 'resnet50':
        model = net.resnet50(pretrained=True)
        model.fc = nn.Linear(2048, nb_classes)
        model.load_state_dict(torch.load('net/finetuned_' + dataset + '_' + net_type + '.pth'))

    elif net_type == 'resnet101':
        model = net.resnet101(pretrained=True)
        model.fc = nn.Linear(2048, nb_classes)
        model.load_state_dict(torch.load('net/finetuned_' + dataset + '_' + net_type + '.pth'))

    elif net_type == 'resnet152':
        model = net.resnet152(pretrained=True)
        model.fc = nn.Linear(2048, nb_classes)
        model.load_state_dict(torch.load('net/finetuned_' + dataset + '_' + net_type + '.pth'))

    elif net_type == 'densenet121':
        model = net.densenet121(pretrained=True)
        model.classifier = nn.Linear(1024, nb_classes)
        model.load_state_dict(torch.load('net/finetuned_' + dataset + '_' + net_type + '.pth'))
        # model.load_state_dict(torch.load('sop_resnet_new/Stanford_paramRes_16bit_densenet121_0.0002_5.2724883734490575e-12_10_6_2.pth'))

    elif net_type == 'densenet161':
        model = net.densenet161(pretrained=True)
        model.classifier = nn.Linear(2208, nb_classes)
        model.load_state_dict(torch.load('net/finetuned_' + dataset + '_' + net_type + '.pth'))
        # model.load_state_dict(torch.load('sop_resnet_new/Stanford_paramRes_16bit_densenet161_0.0002_5.2724883734490575e-12_10_6_2.pth'))

    elif net_type == 'densenet169':
        model = net.densenet169(pretrained=True)
        model.classifier = nn.Linear(1664, nb_classes)
        model.load_state_dict(torch.load('net/finetuned_' + dataset + '_' + net_type + '.pth'))
        # model.load_state_dict(torch.load('sop_resnet_new/Stanford_paramRes_16bit_densenet169_0.0002_5.2724883734490575e-12_10_6_2.pth'))

    elif net_type == 'densenet201':
        model = net.densenet201(pretrained=True)
        model.classifier = nn.Linear(1920, nb_classes)
        model.load_state_dict(torch.load('net/finetuned_' + dataset + '_' + net_type + '.pth'))
        # model.load_state_dict(torch.load('sop_resnet_new/Stanford_paramRes_16bit_densenet201_0.0002_5.2724883734490575e-12_10_6_2.pth'))
    return model



Overwriting net/load_trained_net.py


In [10]:
#@title add get_W_gt to get the ,,groundtruth'' for W
%%writefile data_utility.py

import dataset
import torch
from collections import defaultdict
from combine_sampler import CombineSampler, CombineSamplerAdvanced, CombineSamplerSuperclass, CombineSamplerSuperclass2
import numpy as np
import pickle


def create_loaders(data_root, num_classes, is_extracted, num_workers, num_classes_iter, num_elements_class, size_batch):
    Dataset = dataset.Birds(
        root=data_root,
        labels=list(range(0, num_classes)),
        is_extracted=is_extracted,
        transform=dataset.utils.make_transform())

    ddict = defaultdict(list)
    for idx, label in enumerate(Dataset.ys):
        ddict[label].append(idx)

    list_of_indices_for_each_class = []
    for key in ddict:
        list_of_indices_for_each_class.append(ddict[key])

    dl_tr = torch.utils.data.DataLoader(
        Dataset,
        batch_size=size_batch,
        shuffle=False,
        sampler=CombineSampler(list_of_indices_for_each_class, num_classes_iter, num_elements_class),
        num_workers=num_workers,
        drop_last=True,
        pin_memory=True
    )

    if data_root == 'Stanford':
        class_end = 2 * num_classes - 2
    else:
        class_end = 2 * num_classes

    dl_ev = torch.utils.data.DataLoader(
        dataset.Birds(
            root=data_root,
            labels=list(range(num_classes, class_end)),
            is_extracted=is_extracted,
            transform=dataset.utils.make_transform(is_train=False)
        ),
            batch_size=50,
            shuffle=False,
            num_workers=1,
            pin_memory=True
    )

    dl_finetune = torch.utils.data.DataLoader(
        dataset.Birds(
            root=data_root,
            labels=list(range(num_classes)),
            is_extracted=is_extracted,
            transform=dataset.utils.make_transform(is_train=False)
        ),
        batch_size=size_batch,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    dl_train_evaluate = torch.utils.data.DataLoader(
        dataset.Birds(
            root=data_root,
            labels=list(range(num_classes)),
            is_extracted=is_extracted,
            transform=dataset.utils.make_transform(is_train=False)
        ),
        batch_size=150,
        shuffle=False,
        num_workers=1,
        pin_memory=True
    )

    return dl_tr, dl_ev, dl_finetune, dl_train_evaluate


def create_loaders_finetune(data_root, num_classes, is_extracted, num_workers, size_batch):

    if data_root == 'Stanford':
        class_end = 2 * num_classes - 2
    else:
        class_end = 2 * num_classes

    dl_ev = torch.utils.data.DataLoader(
        dataset.Birds(
            root=data_root,
            labels=list(range(num_classes, class_end)),
            is_extracted=is_extracted,
            transform=dataset.utils.make_transform(is_train=False)
        ),
            batch_size=150,
            shuffle=False,
            num_workers=1,
            pin_memory=True
    )

    dl_finetune = torch.utils.data.DataLoader(
        dataset.Birds(
            root=data_root,
            labels=list(range(num_classes)),
            is_extracted=is_extracted,
            transform=dataset.utils.make_transform(is_train=False)
        ),
        batch_size=size_batch,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )


    return dl_ev, dl_finetune




def create_loaders_new(data_root, num_classes, is_extracted, dict_class_distances, num_classes_iter=0, num_elements_class=0, iterations_for_epoch=200):
    Dataset = dataset.Birds(
       root=data_root,
       labels=list(range(0, num_classes)),
       is_extracted=is_extracted,
       transform=dataset.utils.make_transform())
    ddict = defaultdict(list)
    for idx, label in enumerate(Dataset.ys):
       ddict[label].append(idx)
    list_of_indices_for_each_class = []
    for key in ddict:
       list_of_indices_for_each_class.append(ddict[key])
    # with open('list_of_indices_for_each_class_stanford.pickle', 'wb') as handle:
    #     pickle.dump(list_of_indices_for_each_class, handle, protocol=pickle.HIGHEST_PROTOCOL)
    list_of_indices_for_each_class = pickle.load(open('list_of_indices_for_each_class_stanford.pickle', 'rb'))
    dl_tr = torch.utils.data.DataLoader(
       Dataset,
       batch_size=num_classes_iter * num_elements_class,
       shuffle=False,
       sampler=CombineSamplerSuperclass2(list_of_indices_for_each_class, num_classes_iter, num_elements_class, dict_class_distances, iterations_for_epoch=iterations_for_epoch),
       num_workers=8,
       drop_last=True,
       pin_memory=True
    )
    return dl_tr


def get_labeled_and_unlabeled_points(labels, num_points_per_class, num_classes=100):
    labs, L, U = [], [], []
    labs_buffer = np.zeros(num_classes)
    num_points = labels.shape[0]
    for i in range(num_points):
        if labs_buffer[labels[i]] == num_points_per_class:
            U.append(i)
        else:
            L.append(i)
            labs.append(labels[i])
            labs_buffer[labels[i]] += 1
    return labs, L, U


def get_W_gt(labels):
    n=labels.shape[0]
    W = torch.zeros(n, n)
    for i, yi in enumerate(labels):
        for j, yj in enumerate(labels[i:],i):
              W[i, j] = W[j, i] = yi==yj    
    W = W.cuda()
    W.requires_grad=False
    return W


def debug_info(gtg, model):
    for name, param in gtg.named_parameters():
        if param.requires_grad:
            if param.grad is not None:
                # print(name, param.grad.data.norm(2))
                print(name, torch.mean(param.grad.data))
    for name, param in model.named_parameters():
        if param.requires_grad:
            if param.grad is not None:
                # print(name, param.grad.data.norm(2))
                print(name, torch.mean(param.grad.data))
    print("\n\n\n")



Overwriting data_utility.py


In [11]:
%%writefile similarity.py
import torch
import torch.nn as nn

class learned_similarity(nn.Module):
    def __init__(self, in_size):
        super(learned_similarity, self).__init__()
        #self.bn1 = nn.BatchNorm1d(2*in_features.shape[1])
        self.lin = nn.Linear(in_size, 256)
        self.lin2 = nn.Linear(256, 200)
        self.lin3= nn.Linear(200,1)
        self.relu1= nn.ReLU(False)
        self.relu2=nn.ReLU(False)
        self.relu3=nn.ReLU(False)
        self.current_epoch=0
        #self.sigmoid = nn.Sigmoid()

    def forward(self, xi, xj):
        out = torch.abs(xi - xj)
        #out= self.bn1(out)
        #out= torch.cat((xi, xj), 0)
        out = self.lin(out)
        out= self.relu1(out)
        out= self.lin2(out)
        out= self.relu2(out)
        out= self.lin3(out)
        out = self.relu3(out)
        return out



Writing similarity.py


In [18]:
%%writefile train.py

#from apex import amp
import logging, imp
import random
import os
import sys
import warnings
import argparse

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

import gtg
import net
import data_utility
import utils
from RAdam import RAdam

import argparse
import random
import similarity



def rnd(lower, higher):
    exp = random.randint(-higher, -lower)
    base = 0.9 * random.random() + 0.1
    return base * 10 ** exp


warnings.filterwarnings("ignore")


class Hyperparameters():
    def __init__(self, dataset_name='cub'):
        self.dataset_name = dataset_name
        if dataset_name == 'cub':
            self.dataset_path = '../../datasets/CUB_200_2011'
        elif dataset_name == 'cars':
            self.dataset_path = 'datasets/CARS'  #'../../datasets/CARS'
        else:
            self.dataset_path = '../../datasets/Stanford'
        self.num_classes = {'cub': 100, 'cars': 98, 'Stanford': 11318}
        self.num_classes_iteration = {'cub': 6, 'cars': 5, 'Stanford': 10}
        self.num_elemens_class = {'cub': 9, 'cars': 7, 'Stanford': 6}
        self.get_num_labeled_class = {'cub': 2, 'cars': 3, 'Stanford': 2}
        self.learning_rate = 0.0002
        self.weight_decay = {'cub': 6.059722614369727e-06, 'cars': 4.863656728256105e-07, 'Stanford': 5.2724883734490575e-12}
        self.softmax_temperature = {'cub': 24, 'cars': 79, 'Stanford': 54}

    def get_path(self):
        return self.dataset_path

    def get_number_classes(self):
        return self.num_classes[self.dataset_name]

    def get_number_classes_iteration(self):
        return self.num_classes_iteration[self.dataset_name]

    def get_number_elements_class(self):
        return self.num_elemens_class[self.dataset_name]

    def get_number_labeled_elements_class(self):
        return self.get_num_labeled_class[self.dataset_name]

    def get_learning_rate(self):
        return self.learning_rate

    def get_weight_decay(self):
        return self.weight_decay[self.dataset_name]

    def get_epochs(self):
        return 60

    def get_num_gtg_iterations(self):
        return 1

    def get_softmax_temperature(self):
        return self.softmax_temperature[self.dataset_name]


parser = argparse.ArgumentParser(description='Training inception V2' +
                                             ' (BNInception) on CUB-200-2011 (cub), CARS 196 (cars) and Stanford Online Products (Stanford) with The Group Loss as described in ' +
                                             '`The Group Loss for Deep Metric Learning.`')
dataset_name = 'cars'  # cub, cars or Stanford
parser.add_argument('--dataset_name', default=dataset_name, type=str, help='The name of the dataset')
hyperparams = Hyperparameters(dataset_name)
parser.add_argument('--cub-root', default=hyperparams.get_path(), help='Path to dataset folder')
parser.add_argument('--cub-is-extracted', action='store_true',
                    default=True, help='If `images.tgz` was already extracted, do not extract it again.' +
                                       ' Otherwise use extracted data.')
parser.add_argument('--embedding-size', default=512, type=int, dest='sz_embedding', help='The embedding size')
parser.add_argument('--nb_classes', default=hyperparams.get_number_classes(), type=int,
                    help='Number of first [0, N] classes used for training and ' +
                         'next [N, N * 2] classes used for evaluating with max(N) = 100.')
parser.add_argument('--num_classes_iter', default=hyperparams.get_number_classes_iteration(), type=int,
                    help='Number of classes in the minibatch')
parser.add_argument('--num_elements_class', default=hyperparams.get_number_elements_class(), type=int,
                    help='Number of samples per each class')
parser.add_argument('--num_labeled_points_class', default=hyperparams.get_number_labeled_elements_class(), type=int,
                    help='Number of labeled samples per each class')
parser.add_argument('--lr-net', default=hyperparams.get_learning_rate(), type=float, help='The learning rate')
parser.add_argument('--weight-decay', default=hyperparams.get_weight_decay(), type=float, help='The l2 regularization strength')
parser.add_argument('--nb_epochs', default=hyperparams.get_epochs(), type=int, help='Number of training epochs.')
parser.add_argument('--nb_workers', default=4, type=int, help='Number of workers for dataloader.')
parser.add_argument('--net_type', default='bn_inception', type=str, choices=['bn_inception', 'densenet121', 'densenet161', 'densenet169', 'densenet201',
                                                                            'resnet18', 'resnet34', 'resenet50', 'resnet101', 'resnet152'],
                                                                            help='The type of net we want to use')
parser.add_argument('--sim_type', default='correlation', type=str, help='type of similarity we want to use')
parser.add_argument('--use_w_loss', default=False, action='store_true', help='use a loss on similarity with ,,ground truth W'' ')
parser.add_argument('--set_negative', default='hard', type=str,
                    help='type of threshold we want to do'
                         'hard - put all negative similarities to 0'
                         'soft - subtract minus (-) negative from each entry')
parser.add_argument('--num_iter_gtg', default=hyperparams.get_num_gtg_iterations(), type=int, help='Number of iterations we want to do for GTG')
parser.add_argument('--embed', default=1, type=int, help='boolean controling if we want to do embedding or not')
parser.add_argument('--scaling_loss', default=1, type=int, dest='scaling_loss', help='Scaling parameter for the loss')
parser.add_argument('--scaling_loss_2', default=1/10, type=int, dest='scaling_loss_2', help='Scaling parameter for the loss')
parser.add_argument('--temperature', default=hyperparams.get_softmax_temperature(), help='Temperature parameter for the softmax')
parser.add_argument('--decrease_learning_rate', default=10., type=float,
                    help='Number to divide the learnign rate with')
parser.add_argument('--id', default=1, type=int,
                    help='id, in case you run multiple independent nets, for example if you want an ensemble of nets')
parser.add_argument('--is_apex', default=0, type=int,
                    help='if 1 use apex to do mixed precision training')

args = parser.parse_args()
print(args)

file_name = args.dataset_name + str(args.id) + '_' + args.net_type + '_' + str(args.lr_net) + '_' + str(args.weight_decay) + '_' + str(
    args.num_classes_iter) + '_' + str(args.num_elements_class) + '_' + str(args.num_labeled_points_class)

file_name_2 = args.dataset_name + str(args.id) + '_' + args.net_type + '_' + str(args.lr_net) + '_' + str(args.weight_decay) + '_' + str(
    args.num_classes_iter) + '_' + str(args.num_elements_class) + '_' + str(args.num_labeled_points_class) + '_' + str(args.use_w_loss)+'_'+ str(args.scaling_loss_2)


batch_size = args.num_classes_iter * args.num_elements_class
device = 'cuda:0'

# create folders where we save the trained nets and we put the results
save_folder_nets = 'save_trained_nets'
save_folder_results = 'save_results'
if not os.path.exists(save_folder_nets):
    os.makedirs(save_folder_nets)
if not os.path.exists(save_folder_results):
    os.makedirs(save_folder_results)


# load the pre-trained 
model = net.load_net(dataset=args.dataset_name, net_type=args.net_type, nb_classes=args.nb_classes, embed=args.embed, sz_embedding=args.sz_embedding)

# define the loss and optimizer and put them to cuda
model = model.to(device)
learned_similarity_mod = similarity.learned_similarity(args.sz_embedding).to(device)
gtg = gtg.GTG(args.nb_classes, max_iter=args.num_iter_gtg, sim=args.sim_type,
              sim_model=learned_similarity_mod, set_negative=args.set_negative, device=device).to(device)
opt = RAdam([
             {'params': list(set(model.parameters()))},
             {'params': list(set(learned_similarity_mod.parameters())), 'lr': args.lr_net}
              ],lr=args.lr_net ,weight_decay=args.weight_decay)

criterion_NLL = nn.NLLLoss().to(device)
criterion_CE = nn.CrossEntropyLoss().to(device)
criterion_MSE = nn.MSELoss().to(device)

# do training in mixed precision
if args.is_apex:
    model, opt = amp.initialize(model, opt, opt_level="O1")

# create loaders
dl_tr, dl_ev, _, _ = data_utility.create_loaders(args.cub_root, args.nb_classes, args.cub_is_extracted,
                                                                 args.nb_workers,
                                                                 args.num_classes_iter, args.num_elements_class,
                                                                 batch_size)

# train and evaluate the net
best_accuracy = 0
scores = []
for e in range(29, args.nb_epochs + 1):
    if e == 30:
        model.load_state_dict(torch.load(os.path.join(save_folder_nets, file_name + '.pth')))
        if(args.sim_type=='learnt'):
            learned_similarity_mod.load_state_dict(torch.load(os.path.join(save_folder_nets, file_name_2 + '.pth')))
        for g in opt.param_groups:
            g['lr'] = args.lr_net / 10.

    i = 0
    for x, Y in dl_tr:
        Y = Y.to(device)
        opt.zero_grad()

        probs, fc7 = model(x.to(device))
        labs, L, U = data_utility.get_labeled_and_unlabeled_points(labels=Y,
                                                                   num_points_per_class=args.num_labeled_points_class,
                                                                   num_classes=args.nb_classes)

        # compute the smoothed softmax
        probs_for_gtg = F.softmax(probs / args.temperature)


        # do GTG (iterative process)
        probs_for_gtg, W = gtg(fc7, fc7.shape[0], labs, L, U, probs_for_gtg)

        probs_for_gtg = torch.log(probs_for_gtg + 1e-12)


        # compute the losses
        loss_NLL = criterion_NLL(probs_for_gtg, Y)
        loss_CE = criterion_CE(probs, Y)
        if(args.use_w_loss):
          W_gt=data_utility.get_W_gt(labels=Y)
          loss_W_MSE = criterion_MSE(W_gt,W)
          loss =  args.scaling_loss * loss_NLL + loss_CE+ .1*loss_W_MSE
        else:
          loss= args.scaling_loss * loss_NLL + loss_CE
        i += 1

        # check possible net divergence
        if torch.isnan(loss):
            print("We have NaN numbers, closing")
            print("\n\n\n")
            sys.exit(0)

        # backprop
        if args.is_apex:
            with amp.scale_loss(loss, opt) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()

            
        opt.step()

    # compute recall and NMI at the end of each epoch (for Stanford NMI takes forever so skip it)
    with torch.no_grad():
        logging.info("**Evaluating...**")
        nmi, recall = utils.evaluate(model, dl_ev, args.nb_classes, args.net_type, dataroot=args.dataset_name)
        print('recall:' + str(recall) + 'loss'+str(loss))
        scores.append((nmi, recall))
        model.current_epoch = e
        learned_similarity_mod.current_epoch=e
        if recall[0] > best_accuracy:
            best_accuracy = recall[0]
            torch.save(model.state_dict(), os.path.join(save_folder_nets, file_name + '.pth'))
            if(args.sim_type=='learnt'):
                torch.save(learned_similarity_mod.state_dict(), os.path.join(save_folder_nets, file_name_2 + '.pth'))

with open(os.path.join(save_folder_results, file_name + '.txt'), 'a+') as fp:
    fp.write(file_name + "\n")
    fp.write(str(args))
    fp.write('\n')
    fp.write('\n'.join('%s %s' % x for x in scores))
    fp.write("\n\n\n")


Overwriting train.py


In [13]:

%%writefile gtg.py

import torch.nn as nn
import torch
import dynamics
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F



class GTG(nn.Module):
    def __init__(self, total_classes, tol=-1., max_iter=5, sim='correlation',sim_model=None, set_negative='hard', mode='replicator', device='cuda:0'):
        super(GTG, self).__init__()
        self.m = total_classes
        self.tol = tol
        self.max_iter = max_iter
        self.mode = mode
        self.sim = sim
        self.set_negative = set_negative
        self.device = device
        self.sim_model=sim_model

    def _init_probs_uniform(self, labs, L, U):
        """ Initialized the probabilities of GTG from uniform distribution """
        n = len(L) + len(U)
        ps = torch.zeros(n, self.m).to(self.device)
        ps[U, :] = 1. / self.m
        ps[L, labs] = 1.

        # check if probs sum up to 1.
        assert torch.allclose(ps.sum(dim=1), torch.ones(n))
        return ps

    def _init_probs_prior(self, probs, labs, L, U):
        """ Initiallized probabilities from the softmax layer of the CNN """
        n = len(L) + len(U)
        ps = torch.zeros(n, self.m).to(self.device)
        ps[U, :] = probs[U, :]
        ps[L, labs] = 1.

        # check if probs sum up to 1.
        assert torch.allclose(ps.sum(dim=1), torch.ones(n).cuda())
        return ps

    def _init_probs_prior_only_classes(self, probs, labs, L, U, classes_to_use):
        """ Different version of the previous version when it considers only classes in the minibatch,
            surprisingly it works worse than the version that considers all classes """
        n = len(L) + len(U)
        ps = torch.zeros(n, self.m).to(self.device)
        ps[U, :] = probs[torch.meshgrid(torch.tensor(U), torch.from_numpy(classes_to_use))]
        ps[L, labs] = 1.
        ps /= ps.sum(dim=ps.dim() - 1).unsqueeze(ps.dim() - 1)
        return ps

    def set_negative_to_zero(self, W):
        return F.relu(W)

    def set_negative_to_zero_soft(self, W):
        """ It shifts the negative probabilities towards the positive regime """
        n = W.shape[0]
        minimum = torch.min(W)
        W = W - minimum
        W = W * (torch.ones((n, n)).to(self.device) - torch.eye(n).to(self.device))
        return W

    def _get_W(self, x):

        if self.sim == 'correlation':
            x = (x - x.mean(dim=1).unsqueeze(1))
            norms = x.norm(dim=1)
            W = torch.mm(x, x.t()) / torch.ger(norms, norms)
        elif self.sim == 'cosine':
            W = torch.mm(x, x.t())
        elif self.sim == 'learnt':
            n = x.shape[0]
            W = torch.zeros(n, n)
            for i, xi in enumerate(x):
                for j, xj in enumerate(x[(i ):], i):
                      W[i, j] = W[j, i] = self.sim_model(xi, xj)      
            W = W.cuda()
        if self.set_negative == 'hard':
            W = self.set_negative_to_zero(W.cuda())
        else:
            W = self.set_negative_to_zero_soft(W)
        return W


    def forward(self, fc7, num_points, labs, L, U, probs=None, classes_to_use=None):
        W = self._get_W(fc7)
        if type(probs) is type(None):
            ps = self._init_probs(labs, L, U).cuda()
        else:
            if type(classes_to_use) is type(None):
                ps = probs
                ps = self._init_probs_prior(ps, labs, L, U)
            else:
                ps = probs
                ps = self._init_probs_prior(ps, labs, L, U)
        ps = dynamics.dynamics(W, ps, self.tol, self.max_iter, self.mode)
        return ps, W


Overwriting gtg.py


open console with strg+shift+i and copy the script in there to stay connected to runtime during training.

function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)



In [19]:
!python train.py

Namespace(cub_is_extracted=True, cub_root='datasets/CARS', dataset_name='cars', decrease_learning_rate=10.0, embed=1, id=1, is_apex=0, lr_net=0.0002, nb_classes=98, nb_epochs=60, nb_workers=4, net_type='bn_inception', num_classes_iter=5, num_elements_class=7, num_iter_gtg=1, num_labeled_points_class=3, scaling_loss=1, scaling_loss_2=0.1, set_negative='hard', sim_type='correlation', sz_embedding=512, temperature=79, use_w_loss=False, weight_decay=4.863656728256105e-07)
recall:[0.564260238593039, 0.6769155085475341, 0.7745664739884393, 0.8530316074283606]losstensor(4.8777, device='cuda:0', grad_fn=<AddBackward0>)
recall:[0.5979584306973312, 0.7091378674209814, 0.799409666707662, 0.8716025089164925]losstensor(3.9755, device='cuda:0', grad_fn=<AddBackward0>)
recall:[0.6128397491083507, 0.7181158529086213, 0.8076497355798795, 0.8788586889681466]losstensor(4.1923, device='cuda:0', grad_fn=<AddBackward0>)
recall:[0.6197269708522937, 0.7266018939859796, 0.8147829295289633, 0.879596605583569]lo